# Jupyter Notebook Guide - Dog Breed Classification Using PyTorch, Azure ML, and Visual Studio Code

## Introduction

Have you ever seen a dog and not been able to tell the breed? Some dogs look so similar, that it can be nearly impossible to tell. For instance these are a few breeds that are difficult to tell apart:

#### Alaskan Malamutes vs Siberian Huskies
![Image of Alaskan Malamute vs Siberian Husky](http://cdn.akc.org/content/article-body-image/malamutehusky.jpg)

#### Whippet vs Italian Greyhound 
![Image of Whippet vs Italian Greyhound](http://cdn.akc.org/content/article-body-image/whippetitalian.jpg)

There are sites like https://www.bing.com/visualsearch/Microsoft/WhatDog, which use Microsoft Cognitive Services to be able to make this easier. 

In this tutorial, you will learn how to train your own image classification model using [transfer learning](https://cs231n.github.io/transfer-learning/). The Azure Machine Learning Python SDK's [PyTorch estimator](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-pytorch) enables you to easily submit PyTorch training jobs for both single-node and distributed runs on Azure compute. The model is trained to classify dog breeds using a pretrained ResNet18 model that has been trained on the [Stanford Dog dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/). This dataset has been built using images and annotation from ImageNet for the task of fine-grained image categorization.

In the interest of time, we will use a subset of this dataset which includes 10 dog breeds.

## What is Azure Machine Learning service?

Azure Machine Learning Service is a cloud service that you can use to develop and deploy machine learning models. Using Azure Machine Learning service, you can track your models as you build, train, deploy, and manage them, all at the broad scale that the cloud provides.
![](https://docs.microsoft.com/en-us/azure/machine-learning/service/media/overview-what-is-azure-ml/aml.png)

## Prerequisites
1. Sign up for an [Azure account](https://azure.microsoft.com/en-us/free/).
    * In this workshop, you can choose to use your own account or use the Azure AD credentials provided to you.
2. Verify that the [Azure Machine Learning SDK](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/intro?view=azure-ml-py) is installed by running the following cell.

In [ ]:
# Check Azure ML core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)

# Train image classification models
Training machine learning models, particularly deep neural networks, is often a time- and compute-intensive task. Once you've finished writing your training script and running on a small subset of data on your local machine, you will likely want to scale up your workload. The Azure Machine Learning (AML) service enables users to easily train their models in the Azure ecosystem, and it also provides built-in support for popular machine learning frameworks, such as PyTorch and TensorFlow, to simplify the workflow.

### Steps to train models:

1. Connect to an Azure Machine Learning service Workspace 
2. Create a remote compute cluster for training your models
3. Upload your training data to a remote data store
4. Prepare your Python training script
5. Submit a training job to AML

## 1. Connect to an Azure Machine Learning service Workspace
You will need to create or connect to an AML [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) in order to train your models.  Replace any placeholders in the Python code cells below with your relevant details.

**You will be asked to login during this step. Please use the Azure AD credentials provided to you.**

* If you have trouble authenticating, see [this guide](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb) for alternative authencation methods.

In [ ]:
# Run this code if you are creating a new workspace

from azureml.core.workspace import Workspace

subscription_id = "<your-subscription-id>"
resource_group = "DogBreeds"
workspace_name = "DogBreeds"
workspace_region = "<your-region>"  # e.g. useast or australiaeast

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      create_resource_group = True,
                      exist_ok = True)
ws.write_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
# Otherwise, run this code to connect to an existing workspace

from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

## 2. Create a remote compute cluster for training your models
For this tutorial, you will prepare an AML Compute cluster with NC-series GPU virtual machines to use as the [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) to execute your training script on. 

The following code creates a cluster for you if it does not already exist in your workspace. Otherwise, it will skip the cluster creation process.

**Creation of the cluster may take approximately 5 minutes (if min_nodes > 0).** 

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "TrainingCluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6',   # Standard_NC6s_v2
                                                           min_nodes=0,
                                                           max_nodes=2,
                                                           idle_seconds_before_scaledown='300')

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

    # Use the 'status' property to get a detailed status for the current cluster. 
    print(compute_target.status.serialize())

## 3. Upload your training data to a remote data store

In this tutorial, we will use 10 classes of images for training. Each class represents one dog breed and consists ~150 images: 100 are training images for dog breeds, and ~50 are validation images for each class. You can view the data used [here](breeds-10). 

First, download the dataset (located [here](breeds-10.zip) as a zip file) locally to your current directory and extract the files. This will create a folder called breeds-10 with two subfolders train and val that contain the training and validation images, respectively.

In [ ]:
import os
import urllib
from zipfile import ZipFile

download_url = 'https://github.com/clarenceb/GBB-MLOps-Workshop/blob/master/breeds-10.zip'
data_file = './breeds-10.zip'

if os.path.isfile(data_file):
    print(f'Skipping download, {data_file} already exists locally.')
else:
    urllib.request.urlretrieve(download_url, filename=data_file)

    # extract files
    with ZipFile(data_file, 'r') as zip:
        print('extracting files...')
        zip.extractall()
        print('done')
    
    # delete zip file
    #os.remove(data_file)

To make the data accessible for remote training, you will need to upload the data from your local machine to the cloud. AML provides a convenient way to do so via a [Datastore](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data). The datastore provides a mechanism for you to upload/download data, and interact with it from your remote compute targets.

Each workspace is associated with a [default datastore](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data#use-the-default-datastore-in-your-workspace). In this tutorial, we will upload the training data to this default datastore. The following code will upload the training data to the path `./breeds-10` on the default datastore.

**Note: If your data is already stored in Azure, or you download the data as part of your training script, you will not need to do this step.**

In [ ]:
ds = ws.get_default_datastore()
ds.upload(src_dir='./breeds-10', target_path='breeds-10')
print(ds.datastore_type, ds.account_name, ds.container_name)

Now let's get a reference to the path on the datastore with the training data. We can do so using the `path` method. In the next section, we can then pass this reference to our training script's `--data_dir` argument. 

In [ ]:
path_on_datastore = 'breeds-10'
ds_data = ds.path(path_on_datastore)
print(ds_data)

**If you already have a datastore uploaded you can access it through the following code:**

```python
from azureml.core.datastore import Datastore
ds = Datastore.get(ws,"breeds-10")
```

**Download Data (Optional)**

If you are interested in downloading the data locally, you can run `ds.download("./", 'breeds-10')`. This might take several minutes.

## 4. Prepare your training script

### Create a project directory
Create a project directory that will contain all the necessary code from your local machine that you will need access to on the remote compute resource. This includes the training script and any additional files your training script depends on.

In [ ]:
import os

project_folder = './pytorch-dog-breeds-10'
os.makedirs(project_folder, exist_ok=True)

### Prepare training script
In this tutorial, the training script is already provided for you at `pytorch_train_10.py`. In practice, you should be able to take any custom training script as is and run it with AML without having to modify your code.

However, if you would like to use AML's [tracking and metrics](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-track-experiments) capabilities, you will have to add a small amount of AML code inside your training script. 

In `pytorch_train_10.py`, we will log some metrics to our AML run. To do so, we will access the AML `Run` object within the script:

```python
from azureml.core.run import Run
run = Run.get_context()
```
Further within `pytorch_train_10.py`, we log the learning rate and momentum parameters, and the best validation accuracy the model achieves:

```python
run.log('lr', np.float(learning_rate))
run.log('momentum', np.float(momentum))

run.log('best_val_acc', np.float(best_acc))
```

**Note** These metrics can be view in the [Azure portal](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-track-experiments#view-the-experiment-in-the-azure-portal).

Once your script is ready, copy the training script `pytorch_train_10.py` into your project directory.

In [ ]:
import shutil
shutil.copy('pytorch_train_10.py', project_folder)

## 5. Submit a training job to AML
Now that you have your data and training script prepared, you are ready to train on your remote compute cluster. You can take advantage of Azure compute to leverage GPUs to cut down your training time.     

### Create an experiment
An experiment is a grouping of many runs from a specified script. Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this transfer learning PyTorch tutorial. 

**Please enter your own unique name so that you can track your specific runs.**

In [ ]:
from azureml.core import Experiment

#experiment_name = <<ENTER UNIQUE NAME HERE>>
experiment_name = 'pytorch-dogs'
experiment = Experiment(ws, name=experiment_name)

print(experiment)

### Create a PyTorch estimator
The AML SDK's PyTorch [estimator](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-ml-models) enables you to easily submit PyTorch training jobs for both single-node and distributed runs. For more information on the PyTorch estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-pytorch). The following code will define a [single-node](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-ml-models#single-node-training) PyTorch job.  Refer to the docs for how to run [distributed training on a multi-node cluster](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-ml-models#distributed-training-and-custom-docker-images).

In [ ]:
from azureml.train.dnn import PyTorch

script_params = {
    '--data_dir': ds_data.as_mount(),
    '--num_epochs': 10,
    '--output_dir': './outputs'
}

estimator = PyTorch(source_directory=project_folder, 
                    script_params=script_params,
                    compute_target=compute_target, 
                    entry_script='pytorch_train_10.py',
                    use_gpu=True)


The `script_params` parameter is a dictionary containing the command-line arguments to your training script `entry_script`. Please note the following:
- We passed our training data reference `ds_data` to our script's `--data_dir` argument. This will 1) mount our datastore on the remote compute and 2) provide the path to the training data `breeds-10` on our datastore.
- We specified the output directory as `./outputs`. The `outputs` directory is specially treated by AML in that all the content in this directory gets uploaded to your workspace as part of your run history. The files written to this directory are therefore accessible even once your remote run is over. In this tutorial, we will save our trained model to this output directory.

To leverage the Azure VM's GPU for training, we set `use_gpu=True`.

### Submit the training job
Now let's submit the experiment run to the AML service. Note that this call is asynchronous.

In [ ]:
run = experiment.submit(estimator)
print(run.get_details())

### Monitor your run
Once the run is submitted, you can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [ ]:
!pip install --upgrade azureml-widgets

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

**The run will take a few minutes - check the widget above for its status.**

## Deploy the model as a web service to ACI

Once the wiget shows the run is compete, we are ready to deploy this model as a web service for predictions.

In this tutorial, we will:

1. Register the model with AML
2. Deploy the model as a web service to [Azure Container Instances](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where#aci) (suitable for DEV-TEST)

### 1. Register the model with AML
Model registration allows you to store and version your models in the Azure cloud, in your workspace. The model registry makes it easy to organize and keep track of your trained models.

**Please use a unique name for the model**. You will need to edit the `init()` function in the pytorch_score.py file to match the unique name used. Change the line `model_path = Model.get_model_path('dogs')` to use the unique name.

In [ ]:
model = run.register_model(model_name = 'dogbreedmodel', model_path = 'outputs/model.pt')
print(model.name, model.id, model.version, sep = '\t')

In [ ]:
## if you need to re-reference the run object specifically uncomment this section

# from azureml.core import Run
# run = Run(experiment, run_id="") 

### 2. Deploy model as web service to Azure Container Instances (ACI)

Once you have your trained model, you can deploy the model on Azure. You can deploy your trained model as a web service on Azure Container Instances (ACI), Azure Kubernetes Service (AKS), IoT edge device, or Field Programmable Gate Arrays (FPGAs). ACI is generally cheaper than AKS and can be set up in 4-6 lines of code. ACI is the perfect option for testing deployments. Later, when you're ready to use your models and web services for high-scale, production usage, you can deploy them to AKS.

**Note** ACI is intended for DEV-TEST and AKS for production.  AKS has a requirement of 12 cores (virtual CPUs) per cluster.  You can lift this requirement in DEV-TEST like so:

```python
# Attach the cluster to your workgroup. If the cluster has less than 12 virtual CPUs, use the following instead:
# attach_config = AksCompute.attach_configuration(resource_group = resource_group,
#                                         cluster_name = cluster_name,
#                                         cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
```

In this tutorial, we will deploy the model as a web service in [Azure Container Instances](https://docs.microsoft.com/en-us/azure/container-instances/) (ACI). 

For more information on deploying models using Azure ML, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-deploy-and-where).

### Create the scoring script

First, we will create a scoring script that will be invoked by the web service call. Note that the scoring script must have two required functions - required when packaged as a Docker image for deployment:

* `init()`: In this function, you typically load the model into a `global` object. This function is executed only once when the Docker container is started. 
* `run(input_data)`: In this function, the model is used to predict a value based on the input data. The input and output typically use JSON as serialization and deserialization format, but you are not limited to that.

Here's the scoring file that our web service will use this file to predict the breed of the dog in the image. 

**IMPORTANT: update the model name to match yours.**

When writing your own scoring script, don't forget to test it locally first before you go and deploy the web service.

**Note: this script is already part of this repository.  Running the code cell below will overwrite the file with any changes you've made.**

In [ ]:
%%writefile pytorch_score.py
import torch
import torch.nn as nn
from torchvision import transforms
import json
import base64
from io import BytesIO
from PIL import Image

from azureml.core.model import Model


def preprocess_image(image_file):
    """Preprocess the input image."""
    data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(image_file)
    image = data_transforms(image).float()
    image = torch.tensor(image)
    image = image.unsqueeze(0)
    return image


def base64ToImg(base64ImgString):
    base64Img = base64ImgString.encode('utf-8')
    decoded_img = base64.b64decode(base64Img)
    return BytesIO(decoded_img)


def init():
    global model
    model_path = Model.get_model_path('dogbreedmodel')
    model = torch.load(model_path, map_location=lambda storage, loc: storage)
    model.eval()


def run(input_data):
    img = base64ToImg(json.loads(input_data)['data'])
    img = preprocess_image(img)

    # get prediction
    output = model(img)

    classes = ['Chihuahua',
            'Italian_greyhound',
            'whippet',
            'golden_retriever',
            'Shetland_sheepdog',
            'German_shepherd',
            'boxer',
            'Saint_Bernard',
            'malamute',
            'Siberian_husky']
    ## If you try with 20 classes please uncomment this:
#    classes =['Chihuahua',
#             'Italian_greyhound',
#             'whippet',
#             'Yorkshire_terrier',
#             'golden_retriever',
#             'Labrador_retriever',
#             'Shetland_sheepdog',
#             'Border_collie',
#             'German_shepherd',
#             'Bernese_mountain_dog',
#             'boxer',
#             'bull_mastiff',
#             'French_bulldog',
#             'Great_Dane',
#             'Saint_Bernard',
#             'Siberian_husky',
#             'basenji',
#             'pug',
#             'Samoyed',
#             'Pembroke'
    softmax = nn.Softmax(dim=1)
    pred_probs = softmax(model(img)).detach().numpy()[0]
    index = torch.argmax(output, 1)

    result = json.dumps({"label": classes[index], "probability": str(pred_probs[index])})
    return result

### Create environment file
Next, we will need to create an environment file (`myenv.yml`) that specifies all of the scoring script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image by AML. In this case, we need to specify `torch`, `torchvision`, `pillow`, and `azureml-sdk`.

In [ ]:
%%writefile myenv.yml
name: myenv
channels:
  - defaults
dependencies:
  - pip:
    - torch
    - torchvision
    - pillow
    - azureml-core

### Configure the ACI container where the service runs in

In [ ]:
# Configure the container image
from azureml.core.image import ContainerImage
image_config = ContainerImage.image_configuration(execution_script='pytorch_score.py', 
                                                  runtime='python', 
                                                  conda_file='myenv.yml',
                                                  description='Image with dog breed model')
# Configure the ACI container
# Note: This will create a public IP endpoint for your scoring URI.  If you have blocked public IPs this won't work.
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1,
                                               location='<your_preferred_region>', # e.g. useast, westus, australiaeast
                                               tags={'data': 'dog_breeds', 'method':'transfer learning', 'framework':'pytorch'},
                                               description='Classify dog breeds using transfer learning with PyTorch')
# Retrieve the model from your workspace.
from azureml.core.model import Model

model = Model(ws, name='dogbreedmodel')

### Deploy the registered model
Finally, let's deploy a web service from our registered model. 
Then, deploy the web service using the ACI config and image config files created in the previous steps. We pass the `model` object in a list to the `models` parameter. If you would like to deploy more than one registered model, append the additional models to this list.

**Please use a unique service name.**

**Note**: `Webservice.deploy_from_model` creates a new Docker image every time (which can take a long time).  If you have a previously created image that you want to deploy, use: [`WebService.deploy_from_image`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice(class)?view=azure-ml-py#deploy-from-image-workspace--name--image--deployment-config-none--deployment-target-none-)

In [ ]:
%%time
from azureml.core.webservice import Webservice

service_name = 'dogbreedsvc'
service = Webservice.deploy_from_model(workspace=ws,
                                       name=service_name,
                                       models=[model],
                                       image_config=image_config,
                                       deployment_config=aciconfig,)

service.wait_for_deployment(show_output=True)
print(service.state)

**Tip: If your deployment fails for any reason, the first thing to look at is the logs from the service by running the following command: `service.get_logs()`. If you need to redeploy, make sure to delete the service before you do so: `service.delete()`.**

Get the web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [ ]:
print(service.scoring_uri)

## Now let's test our web service!
Finally, let's test our deployed web service. We will send the image data as a base64-encoded JSON string to the web service hosted in ACI and use the SDK's `run` API to invoke the service. Here we will take an arbitrary image from online to predict on (or uncomment a URL below to a specific image). This is the same as above, but now we are testing on our own trained model. You can use any dog image, but please remember we only trained on 10 classes so an uknown class will yield unreliable results.

In [ ]:
# Install skimage, if needed
!pip install --upgrade scikit-image

In [ ]:
%matplotlib inline
# importing the requests library 
import requests 
import os, json, base64
from io import BytesIO
import matplotlib.pyplot as plt
from skimage import io
from PIL import Image
import urllib.request
import io

##Get random dog
def get_random_dog():
    r = requests.get(url ="https://dog.ceo/api/breeds/image/random")
    URL= r.json()['message']
    return URL

##Get Random Dog Image
URL = get_random_dog()

##whippet Example 
#URL="https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12223018/Whippet-On-White-03.jpg"

##italian greyhound Example
# URL="https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12231757/Italian-Greyhound-On-White-03.jpg"

##chihuahua Example
# URL ="https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12213613/Chihuahua-onWhite-13.jpg"

##whippet
# URL="https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12223018/Whippet-On-White-03.jpg"

##italian greyhound
# URL="https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12231757/Italian-Greyhound-On-White-03.jpg"

with urllib.request.urlopen(URL) as url:
    test_img = io.BytesIO(url.read())

# ## If you downloaded the dataset, you can try this arbitrary image from the test dataset
# # test_img = os.path.join('breeds-10', 'val', 'n02085620-Chihuahua', 'n02085620_1271.jpg') 

plt.imshow(Image.open(test_img))

In [ ]:
from azureml.core.webservice import Webservice
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

service_name = 'dogbreedsvc'
service = Webservice(ws, service_name)

In [ ]:
def imgToBase64(img):
    """Convert pillow image to base64-encoded image"""
    imgio = BytesIO()
    img.save(imgio, 'JPEG')
    img_str = base64.b64encode(imgio.getvalue())
    return img_str.decode('utf-8')

base64Img = imgToBase64(Image.open(test_img))

result = service.run(input_data=json.dumps({'data': base64Img}))
print(json.loads(result))

## (Optional) Test the web service from within Visual Studio Code

Now let's write a Python script in Visual Studio Code to call the web service for predicting dog breeds. 

Here are the steps we will go through in VS Code:
1. Import a Jupyter Notebook
2. Explore the AML workspace
3. Test the web service

### 0. Prerequisites

0.1. Visual Studio Code is a free, lightweight, cross-platform code editor. [Install Visual Studio Code](https://code.visualstudio.com/) if you don't have it installed.
![VS Code](screenshots/VSCode.PNG)

0.2. Install the Microsoft Python extension in VS Code.
![Python extension](screenshots/PythonExt.PNG)

0.3. Install the Azure Machine Learning extension in VS Code.
![AML extension](screenshots/AMLExt.PNG)

### 1. Import a Jupyter Notebook

First, [Download this Jupyter Notebook](test-in-vscode.ipynb) onto your local machine. This notebooks contains the same code we used above to test the web service, but in a separate notebook file. Now we will import this notebook file into VS Code.

Launch VS Code, use menu `File->Open Folder` to open the folder where you save the Jupyter Notebook file. 
![Open Folder in VS Code](screenshots/OpenFolder.png)

In the EXPLORER window in VS Code, click on the `test-in-vscode.ipynb` file, and you should see a notification message in the bottom right corner. Choose `Yes` to import the Notebook into Python code.
![Import Jupyter Notebook into Python code in VS Code](screenshots/Import.PNG)

Import takes a few seconds. Once finished, you will have a Python file created like the following:
![Import finished](screenshots/ImportDone.PNG)

### 2. Explore the AML workspace
Before we can test the web service in VS Code, we need to fill in the service uri that we will be testing. The [Azure Machine Learning extension for VS Code](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.vscode-ai) provides an easy way to build, train, and deploy machine learning models to the AML service. Now let's use it to find out what the service uri is for the service we just deployed.

Click on the `Azure` tab in the VS Code Activity bar. Use the AAD credentials provides to you to log in. Expand the `DogBreeds` workspace, and look for your service under `Deployments`. Then right click on your service and choose `View Service Properties`.
![Find service](screenshots/FindService.PNG)

In the Properties page, find the `scoringUri` on line 78. Copy the value to clipboard.
![ScoringUri](screenshots/ScoringURI.PNG)

### 3. Test the web service
Now switch back to the Python file that was imported, paste the `scoringUri` value at line 39.
![Paste ScoringUri in the Python code](screenshots/PasteScoringURI.PNG)

Now we are ready to run the Python code to send a dog image to the web service to predict the dog breed. To better visualize the program results, we are going to run the code in the `Python Interactive` window, which brings the power of Jupyter Notebooks into VS Code.

Scroll to the top of the Python file, and click on the `Run Cell` link. This will launch a Jupyter server on the local machine, which will then be used to execute the code. Results are sent back to VS Code for presentation.
![Run Cell](screenshots/RunCell.PNG)


**If you don't have Jupyter installed on your local machine**, you will be prompted with this message. 
Please follow the steps here: [Installing Jupyter Notebook
](https://jupyter.readthedocs.io/en/latest/install.html).

Once the cell finished running, you will see a dog image along with the code in the `Python Interactive` window: 
![First cell result](screenshots/1stCell.PNG)

Continue to run the second cell, which sends back the prediction result:
![Second cell result](screenshots/2ndcell.PNG)

### Delete web service
Once you no longer need the web service, you should delete it.

In [ ]:
service.delete()

## Clean-up resources
When you have finished with the workshop, you can clean up remaining resources to save on costs.
Delete your Resource Group `DogBreeds` (or the name you chose) from the Azure Portal to remove all Azure Resources created for this workshop.

If you plan to complete [Part 2 - MLOps Guide](./mlops.md) then **do not** delete your resources.